---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [59]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 520

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [173]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

  col1 col2
1    w    a
2   10   30
0   20  NaN
  col1 col2
1    w    a
2    A   30
0    B  NaN


In [41]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    #states = {y:x for x,y in states.items()}
    towns=open('university_towns.txt','r').read().split('\n')
    list_towns=[[]]
    state='';
    region='';
    
    for row in towns:
        if '[edit]' in row:
            state=row.replace('[edit]','')
        else :
            region=row.split('(')[0]
            list_towns.append([state,region])
    
    df_towns=pd.DataFrame(list_towns[1:-1],columns=['State', 'RegionName'])
    df_towns['State']=[x.rstrip() for x in df_towns['State']]
    df_towns['RegionName']=[x.rstrip() for x in df_towns['RegionName']]
#    df_towns=df_town.iloc[!df_towns['RegionName'] in ["The Colleges of Worcester Consortium", "The Five College Region of Western Massachusetts", "North Mankato, South Central College", "Cecil B. Moore, Philadelphia, also known as 'Templetown'"]]
#    df_towns['State'].replace(states,inplace=True)
    return df_towns

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [42]:
# Test for Q1
import re
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))

Shape test:  Passed
Index test:  Passed
Column test:  Passed
\n test:  Passed
Trailing whitespace test: Passed
"(" test: Passed
"[" test: Passed
State test:  Passed
RegionName test:  Passed


In [43]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls')
    GDP=GDP.iloc[GDP.index[GDP.iloc[:,4]=='2001q1'][0]:,[-4,-2]]
    GDP.columns=['Quarterly','GDP in billions of chained 2009 dollars']
    
    recession=''
    start=-1
    for i in range(1,len(GDP)-1):
        if (start==-1) & (GDP.iloc[i-1,1]>GDP.iloc[i,1]>GDP.iloc[i+1,1]):
            start=GDP.iloc[i,0]
            #print(GDP.iloc[i-1,0])
        elif (start!=-1) & (GDP.iloc[i-1,1]<GDP.iloc[i,1]<GDP.iloc[i+1,1]):
            recession=start
            start=-1
    return recession
get_recession_start()

'2008q3'

In [56]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls')
    GDP=GDP.iloc[GDP.index[GDP.iloc[:,4]=='2000q1'][0]:,[-4,-2]]
    GDP.columns=['Quarterly','GDP in billions of chained 2009 dollars']
    #print(GDP)
    
    recession=''
    start=-1
    for i in range(1,len(GDP)-1):
        if (start==-1) & (GDP.iloc[i-1,1]>GDP.iloc[i,1]>GDP.iloc[i+1,1]):
            start=GDP.iloc[i-1,0]
        elif (start!=-1) & (GDP.iloc[i-1,1]<GDP.iloc[i,1]<GDP.iloc[i+1,1]):
            recession=GDP.iloc[i+1,0]
            start=-1
    return recession
get_recession_end()

'2009q4'

In [57]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls')
    GDP=GDP.iloc[GDP.index[GDP.iloc[:,4]=='2000q1'][0]:,[-4,-2]]
    GDP.columns=['Quarterly','GDP in billions of chained 2009 dollars']
    #print(GDP)
    
    recession=''
    start=-1
    for i in range(1,len(GDP)-1):
        if (start==-1) & (GDP.iloc[i-1,1]>GDP.iloc[i,1]>GDP.iloc[i+1,1]):
            start=GDP.iloc[i-1,0]
        elif (start!=-1) & (GDP.iloc[i-1,1]<GDP.iloc[i,1]<GDP.iloc[i+1,1]):
            recession=GDP.iloc[i-1,0]
            start=-1
    return recession
get_recession_bottom()

'2009q2'

In [115]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #pd.options.display.float_format = '${:,.2f}'.format
    #pd.options.display.float_format = '{:,.2f}'.format
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    price=pd.read_csv('City_Zhvi_AllHomes.csv')  
    #print(price.head())
    price.drop(price.columns[0],axis=1,inplace=True)
    start=price.columns.get_loc("2000-01")
    end=price.columns.get_loc("2016-08")
    
    for i in range(start, end,12):
        #print(i)
        if str(price.columns[i])[0:4] == '2016':
            price[str(price.columns[i])[0:4]+'q1']=price.iloc[:,i:i+3].mean(axis=1)
            #print(price.iloc[:,i:i+3].head(),price.iloc[:,i:i+3].mean(axis=1).head())
            price[str(price.columns[i])[0:4]+'q2']=price.iloc[:,i+3:i+6].mean(axis=1)
            #print(price.iloc[:,i+3:i+6].head(),price.iloc[:,i+3:i+6].mean(axis=1).head())
            price[str(price.columns[i])[0:4]+'q3']=price.iloc[:,i+6:i+8].mean(axis=1)
            #print(price.iloc[:,i+6:i+8].head(),price.iloc[:,i+6:i+8].mean(axis=1).head())
        else:
            price[str(price.columns[i])[0:4]+'q1']=price.iloc[:,i:i+3].mean(axis=1)
            #print(price.iloc[:,i:i+3].head(),price.iloc[:,i:i+3].mean(axis=1).head())
            price[str(price.columns[i])[0:4]+'q2']=price.iloc[:,i+3:i+6].mean(axis=1)
            price[str(price.columns[i])[0:4]+'q3']=price.iloc[:,i+6:i+9].mean(axis=1)
            price[str(price.columns[i])[0:4]+'q4']=price.iloc[:,i+9:i+12].mean(axis=1)
            #print(price.iloc[:,i+9:i+12].head(),price.iloc[:,i+9:i+12].mean(axis=1).head())
    price.drop(price.columns[2:end+1],axis=1,inplace=True)
    price.reset_index()
    price = price.replace({'State':states})
    price = price.set_index(['State', 'RegionName'])
    return price
convert_housing_data_to_quarters()
#convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,"515,466.67","522,800.00","528,066.67","532,266.67","540,800.00","557,200.00","572,833.33","582,866.67","591,633.33","587,200.00"
California,Los Angeles,"207,066.67","214,466.67","220,966.67","226,166.67","233,000.00","239,100.00","245,066.67","253,033.33","261,966.67","272,700.00",...,"498,033.33","509,066.67","518,866.67","528,800.00","538,166.67","547,266.67","557,733.33","566,033.33","577,466.67","584,050.00"
Illinois,Chicago,"138,400.00","143,633.33","147,866.67","152,133.33","156,933.33","161,800.00","166,400.00","170,433.33","175,500.00","177,566.67",...,"192,633.33","195,766.67","201,266.67","201,066.67","206,033.33","208,300.00","207,900.00","206,066.67","208,200.00","212,000.00"
Pennsylvania,Philadelphia,"53,000.00","53,633.33","54,133.33","54,700.00","55,333.33","55,533.33","56,266.67","57,533.33","59,133.33","60,733.33",...,"113,733.33","115,300.00","115,666.67","116,200.00","117,966.67","121,233.33","122,200.00","123,433.33","126,933.33","128,700.00"
Arizona,Phoenix,"111,833.33","114,366.67","116,000.00","117,400.00","119,600.00","121,566.67","122,700.00","124,300.00","126,533.33","128,366.67",...,"164,266.67","165,366.67","168,500.00","171,533.33","174,166.67","179,066.67","183,833.33","187,900.00","191,433.33","195,200.00"
Nevada,Las Vegas,"132,600.00","134,366.67","135,400.00","137,000.00","139,533.33","141,733.33","143,366.67","146,133.33","149,333.33","150,933.33",...,"170,066.67","173,400.00","175,466.67","177,500.00","181,600.00","186,766.67","190,633.33","194,600.00","197,200.00","199,950.00"
California,San Diego,"222,900.00","234,366.67","245,433.33","256,033.33","267,200.00","276,266.67","284,500.00","291,933.33","301,233.33","312,866.67",...,"480,200.00","489,033.33","496,433.33","503,366.67","512,066.67","519,766.67","525,466.67","529,333.33","536,233.33","539,750.00"
Texas,Dallas,"84,466.67","83,866.67","84,866.67","87,833.33","89,733.33","89,300.00","89,066.67","90,900.00","92,566.67","93,800.00",...,"106,633.33","108,900.00","111,533.33","113,700.00","121,133.33","128,566.67","134,600.00","140,500.00","144,600.00","149,300.00"
California,San Jose,"374,266.67","406,566.67","431,866.67","455,500.00","470,666.67","470,200.00","456,800.00","445,566.67","441,433.33","457,766.67",...,"679,400.00","697,033.33","714,933.33","731,433.33","756,733.33","776,400.00","789,133.33","803,600.00","818,933.33","822,200.00"


In [142]:
from scipy import stats
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    houses= convert_housing_data_to_quarters()
    university_towns= get_list_of_university_towns()
    university_towns['Unitown'] = True
    
    start=get_recession_start()
    bottom=get_recession_bottom()
    end=get_recession_end()
    houses['decline_growth']=houses[start]-houses[bottom]
    houses=houses.reset_index()
    house_in_unitown = pd.merge(houses, university_towns, how='inner',on = ['State', 'RegionName'])
    houses = pd.merge(houses, house_in_unitown, how='outer', on = ['State', 'RegionName','decline_growth'])
    #print(houses)
    houses['Unitown'] = houses['Unitown'].fillna(False)
       
    unitowns = houses[houses['Unitown'] == True]
    no_unitowns = houses[houses['Unitown'] == False]
    
    t, p = ttest_ind(unitowns['decline_growth'].dropna(), no_unitowns['decline_growth'].dropna())

    better = "university town" if unitowns['decline_growth'].mean() < no_unitowns['decline_growth'].mean() else "non-university town"    
    result=(p<0.01,p,better)
    #print(A.info())
    return result
run_ttest()


(True, 0.0043252148535112009, 'university town')